<a href="https://colab.research.google.com/github/xs912/CIAN_binary_classification/blob/main/CIAN_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Задача**
По каждой из фотографий нужно предсказать, что изображено: фото внутреннего жилого помещения или что-либо снаружи.

# **Требования**
Финальное решение (обучение модели и предсказание на датасете) должно быть
предоставлено в виде ipython-ноутбука или питоновского пакета(Python3.x)
Использовать только стандартные библиотеки питона

In [ ]:
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from glob import glob
from scipy import stats
from PIL import Image

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!chmod 600 '/content/kaggle.json'
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 4.4 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=2f9d84f2792ffe705f899d351171832a76e3901bf600ea2d16ebd8aca156e290
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c cian-datafest-2019

 95% 137M/145M [00:00<00:00, 165MB/s]
100% 145M/145M [00:00<00:00, 152MB/s]


In [ ]:
!unzip cian-datafest-2019.zip

Archive:  cian-datafest-2019.zip
  inflating: cian_datafest.zip       
  inflating: test.zip                
  inflating: train.zip               


In [ ]:
!unzip train.zip
!unzip test.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: test/17529.jpg          
  inflating: test/1270.jpg           
  inflating: test/21143.jpg          
  inflating: test/53906.jpg          
  inflating: test/25045.jpg          
  inflating: test/16587.jpg          
  inflating: test/60122.jpg          
  inflating: test/11404.jpg          
  inflating: test/19175.jpg          
  inflating: test/31046.jpg          
  inflating: test/61474.jpg          
  inflating: test/30956.jpg          
  inflating: test/47761.jpg          
  inflating: test/16638.jpg          
  inflating: test/39949.jpg          
  inflating: test/76167.jpg          
  inflating: test/14326.jpg          
  inflating: test/64958.jpg          
  inflating: test/42782.jpg          
  inflating: test/41237.jpg          
  inflating: test/44253.jpg          
  inflating: test/25771.jpg          
  inflating: test/62214.jpg          
  inflating: test/26706.jpg          
  inflating: test/14

Определим пути для изображений в тренировочной и тестовой выборках

In [ ]:
train_images = glob('./train/*/*.jpg')
test_images = glob ('./test/*.jpg')

In [ ]:
train_images[12], len(train_images)

('./train/indoor/41475.jpg', 55322)

In [ ]:
def feat_extract(img_path, easy = True):
  """
  Функция извлечения признаков из картинок. На вход поступает путь изображения, на выходе массив признаков.
  Для извлечения используются только параметры stats.describe, оказывающие существенное влияние на точность. 
  Использование большого количества параметров может увеличить точность, но при этом увеличивает общее время выполнения ноутбука, что противоречит требованиям.
  """
  features = []
  np_image = np.array(img_path) 
  part1 = np.array_split(np_image, 7,axis = 0)
  part2 = np.array_split(np_image, 7,axis = 1)
  for im_part in [np_image]+part1[:3]+part1[-3:]+part2[:3]+part2[-3:]:  
    for i in range(3):
      if easy:
        features += [im_part[:,:,i].mean(), im_part[:,:,i].std()]
      else:
        stat_metrics = stats.describe(im_part[:,:,i],axis = None)
        features += [stat_metrics.mean, stat_metrics.variance, stat_metrics.minmax[0], stat_metrics.minmax[1], stat_metrics.skewness]
  return features

Создаем массивы признаков для картинок двух классов и определяем массив меток классов по пути к изображениям.

In [ ]:
indoor=[]
outdoor=[]

easy = 0

for i, path in enumerate(train_images):
  image = Image.open(path) 
  image_class = path.split('/')[-2] 
   
  if image_class == 'indoor':
    indoor.append(feat_extract(image, easy))
  else:
    outdoor.append(feat_extract(image, easy))

X = np.array(indoor+outdoor)
y = np.array([0 for i in range(len(indoor))] + [1 for i in range(len(outdoor))])

Выделяем валидационную выборку

In [ ]:
indices = np.random.permutation(X.shape[0])
train_indices, val_indices = indices[:80], indices[80:]
X_val, X_train = X[train_indices,:], X[val_indices,:]
Y_val, Y_train = y[train_indices], y[val_indices]

X_train = X_train / np.linalg.norm(X_train, axis = 0)
y = Y_train

In [ ]:
alpha=0.3
n = X_train.shape[0]
x0 = np.ones((n,1))
beta = np.random.rand(1, X_train.shape[1])
for i in range(24000):
    y_pred = np.dot(X_train, beta.T)
    cost = np.sum((y_pred - Y_train.reshape(n, 1))**2)/n
    diff = (y_pred - Y_train.reshape(n,1))
    derivative = 2 * np.dot(diff.T, X_train)/n
    beta = (beta) - alpha * (derivative)
 
model = np.dot(X_val / np.linalg.norm(X_val, axis = 0),beta.T)
model = (model-model.min())/(model.max()-model.min())

In [ ]:
test=[]
test_indices = []
for i, path in enumerate(test_images):
  image = Image.open(path) 
  imageid = int(path.split('/')[-1].replace('.jpg',''))
  test.append(feat_extract(image, easy))
  test_indices.append(imageid)

In [ ]:
X_test = np.array(test)
test_predict = np.dot(X_test / np.linalg.norm(X_test, axis = 0),beta.T)
test_predict = (test_predict - test_predict.min())/(test_predict.max() - test_predict.min())

In [ ]:
import pandas as pd
df = pd.DataFrame({'image_number':test_indices}).join(pd.DataFrame(test_predict, columns = ['prob_outdoor']))

In [ ]:
df

,image_number,prob_outdoor
0,32908,0.368862
1,44358,0.566130
2,5478,0.410861
3,9977,0.491358
4,3467,0.383188
...,...,...
27655,45758,0.479247
27656,5812,0.450537
27657,81637,0.461351
27658,2457,0.430703


Вывод результатов (вероятностей того, что фотография не относится к помещению) в файл

In [ ]:
df.to_csv('submission.csv',index = False)

Для решения этой задачи можно было конвертировать картинки в цветовое пространство LAB, в качестве признаков использовать статистику для канала L (это может быть полезным при различении изображений с большей и меньшей освещенностью), сверточные фильтры (для выделения вертикальных и горизонтальных линий, в большей степени характерных для уличных изображений), также цветовую гистограмму, а затем полученный массив подать на вход полносвязной нейросети, выполняющей функцию линейной регрессии. К сожалению, данное решение оказалось весьма ресурсоемким и поэтому здесь не использовалось.